In [1]:
print("hi")

hi


### Import requests

In [1]:
!pip install requests --quiet
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
CLIENT_ID = '586ecea45142471cb181007f2ffe630b'
CLIENT_SECRET = '187ee73885934d9d86a78d4965b7d6d7'
# Define the required scopes
SCOPES = 'user-read-recently-played'

### Grab access Token

In [39]:
# Function to authenticate and get the access token
def get_access_token(client_id, client_secret, scopes):
    auth_response = requests.post('https://accounts.spotify.com/api/token',
                                  data={'grant_type': 'client_credentials',
                                        'client_id': client_id,
                                        'client_secret': client_secret,
                                        'scope': scopes}).json()
    return auth_response['access_token']

# Get access token
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET, SCOPES)



In [40]:
# Set headers
HEADERS = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [41]:
# auth_response = requests.post('https://accounts.spotify.com/api/token',
#     {'grant_type': 'client_credentials',
#     'client_id': CLIENT_ID,
#     'client_secret': CLIENT_SECRET}).json()
# access_token = auth_response['access_token']
# HEADERS = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [42]:
print(access_token)

BQCwSMSuNZmCij1TdSVpdhqx8ccDaA-UPl7aiy5qkguGQspt-ibv2DKtcynEVZjNoEw--SYTNqurTw16ehmED3CXz4tL6jnW2pfMDRhXorFgPiIP39OwUq-G


In [43]:
# Idk if we need this one, use different later. not working. need right link? 

In [27]:
def get_listening_history(token, before, limit):
    # get the endpoint url
    url = 'https://api.spotify.com/v1/me/player/recently-played'
    
    # Get the Authorization header for the request
    headers = {'Authorization': f'Bearer {token}'}
    
    # setup the query
    query = f"?before={before}&limit={limit}"
    
    # et the query  url
    query_url = url + query
    
     #Send a GET request to the Spotify API with the query URL and headers
    result = requests.get(query_url, headers=headers)
    print(result)

In [28]:
access = '187ee73885934d9d86a78d4965b7d6d7'
get_listening_history(access, '2024-02-28', 50)

<Response [401]>


### Function that specifies the timestamp of one day

In [44]:
import datetime

# Function to get the start and end timestamps for a specific day
def get_day_window(date):
    date_obj = datetime.datetime.strptime(date, '%Y-%m-%d')  # Convert the date string to a datetime object
    
    # Get the start timestamp for the day (00:00:00)
    start_timestamp = date_obj.replace(hour=0, minute=0, second=0).isoformat() + 'Z'
    
    # Get the end timestamp for the day (23:59:59)
    end_timestamp = date_obj.replace(hour=23, minute=59, second=59).isoformat() + 'Z'
    
    return start_timestamp, end_timestamp

### given a track id, return metadata

In [45]:
def get_track_data(track_id):
    """ Given track_id, returns a dictionary
        containing the track's catalog information
        (e.g. name, artists, genres) and audio feature
        metadata (e.g. tempo, key, energy).
    """
    # Get each track's audio features
    audio = requests.get(BASE_URL + 'audio-features/' + track_id,
                         headers=HEADERS).json()
    # Get each track's catalog information
    track = requests.get(BASE_URL + 'tracks/' + track_id,
                         headers=HEADERS).json()
    artists = []
    genres = []
    for artist in track['artists']:
        artist_id = artist['id']
        artist_genres = requests.get(BASE_URL + 'artists/' + artist_id,
                                     headers=HEADERS).json()['genres']
        artists.append(artist['name'])
        genres.append(artist_genres)
    genres = [item for sublist in genres for item in sublist] # Flatten list
    
    # Combine into one dictionary and return
    audio.update({
        'name': track['name'],
        'artists': artists,
        'genres': genres,
        'popularity': track['popularity'],
        'explicit': track['explicit']
    })
    return audio

In [46]:
sample_track_id = '1Ist6PR2BZR3n2z2Y5R6S1'
get_track_data(sample_track_id)

{'danceability': 0.566,
 'energy': 0.647,
 'key': 1,
 'loudness': -6.815,
 'mode': 0,
 'speechiness': 0.0335,
 'acousticness': 0.0477,
 'instrumentalness': 0,
 'liveness': 0.121,
 'valence': 0.22,
 'tempo': 111.96,
 'type': 'audio_features',
 'id': '1Ist6PR2BZR3n2z2Y5R6S1',
 'uri': 'spotify:track:1Ist6PR2BZR3n2z2Y5R6S1',
 'track_href': 'https://api.spotify.com/v1/tracks/1Ist6PR2BZR3n2z2Y5R6S1',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Ist6PR2BZR3n2z2Y5R6S1',
 'duration_ms': 264773,
 'time_signature': 4,
 'name': 'Summertime Sadness',
 'artists': ['Lana Del Rey'],
 'genres': ['art pop', 'pop'],
 'popularity': 76,
 'explicit': False}

In [ ]:
# trying January
for day in range(1, 31):  # Assuming a month has 30 days
    date_to_fetch = f'2024-01-{day:02d}'  
    start_time, end_time = get_day_window(date_to_fetch)
    
    print(f"Fetching data for {date_to_fetch}:")
    
    # Retrieve tracks listened to on the specified day
    result = requests.get('https://api.spotify.com/v1/me/player/recently-played',
                          headers=HEADERS,
                          params={'before': end_time, 'limit': 50}).json()
    print(result)
#     # get the endpoint url
#     url = 'https://api.spotify.com/v1/me/player/recently-played'
    
#     # Get the Authorization header for the request
#     headers = {'Authorization': f'Bearer {token}'}
    
#     # setup the query
#     query = f"?before={before}&limit={limit}"
    
#     # et the query  url
#     query_url = url + query
    
#      #Send a GET request to the Spotify API with the query URL and headers
#     result = requests.get(query_url, headers=headers)
#     result.json()['items'] # i added this but lowkey this might not go here

Fetching data for 2024-01-01:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-02:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-03:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-04:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-05:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-06:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-07:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-08:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-09:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-10:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-11:
{'error': {'status': 500, 'message': 'Server error.'}}
Fetching data for 2024-01-12:
{'error': {'status': 500, 'message'